# Water Network Analysis

In [ ]:
import logging
import warnings
import pandas as pd
import json
from pyincore import InsecureIncoreClient
from pyincore.analyses.waternetworkdamage import WaterNetworkDamage
from pyincore.analyses.waternetworkrecovery import WaterNetworkRecovery, WaterNetworkRecoveryUtil
from pyincore.analyses.stochastic_population import StochasticPopulationAllocation
from pyincore.analyses.populationdislocation import PopulationDislocation, PopulationDislocationUtil
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore import InventoryDataset
from matplotlib import animation, rc
from IPython.display import HTML
import csv
from collections import OrderedDict

#### embed interactive graphs in the notebook

In [ ]:
from plotly.offline import init_notebook_mode
init_notebook_mode()

#### supress warning and info logging

In [ ]:
warnings.filterwarnings('ignore')
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

## Preparation work
 - extract and save demand node
 - use QGIS to generate voronoi cell
 - use QGIS to do clipping
 - use QGIS to calculate the cell area (unit: acres)
 - use that voronoi cell shapefile and building inventory, find the mapping relationship of those two

In [ ]:
def animate(i, resultsNday, wnNday):
    return wn_recovery.output_water_network(resultsNday, 
                                            wnNday, 
                                            timestamp=3600*i,
                                            plotly_options={'title': str(i) +' hours',
                                                 'figsize':[800, 450],
                                                 'node_cmap':'RdBu'
                                             }
                                           )

In [ ]:
pd.read_csv('waternode_building_relations.csv')

## Time 0 : before hazard happens
### run stochastic allocation model 

In [ ]:
seed_i = 1111
addres_inv_path = "IN-CORE_01av3_SetupSeaside_FourInventories_2018-08-29_addresspointinventory.csv"
blding_inv_path = "IN-CORE_01av3_SetupSeaside_FourInventories_2018-08-29_buildinginventory.csv"
infras_inv_path = "IN-CORE_01av3_SetupSeaside_FourInventories_2018-08-29_waterinventory.csv"
popula_inv_path = "IN-CORE_01av3_SetupSeaside_FourInventories_2018-08-29_popinventory.csv"
intermediate_files = True
stal = StochasticPopulationAllocation(addres_inv_path, blding_inv_path,
                                     infras_inv_path, popula_inv_path,
                                     "pop_allocation", "./",
                                     1111, 1,
                                     intermediate_files)
stal.get_stochastic_population_allocation()

### calculate inital water demand based on stochastic allocation model  $unit = m^3/s$

In [ ]:
waternode_population = \
WaterNetworkRecoveryUtil.calc_waternode_population('waternode_building_relations.csv', 
                                                   'pop_allocation_1111.csv')
waternode_population

In [ ]:
demand0 = WaterNetworkRecoveryUtil.calc_waternode_demand('waternode_building_relations.csv', 
                                                               waternode_population)
demand0

### construct the water network with initial demand ratio

In [ ]:
WN_input_file = 'SEASIDE/skeletonized_WN/Seaside_Skeletonized_WN.inp'
wn_recovery = WaterNetworkRecovery(WN_input_file, demand0)

### plot the initial water network (OPTIONAL)

In [ ]:
import pickle
import wntr
with open('wn.pickle', 'rb') as f:
    wn = pickle.load(f)

wn.options.time.duration = 24 * 3600
wn.options.time.hydraulic_timestep = 3600
wn.options.time.report_timestep = 3600
sim = wntr.sim.WNTRSimulator(wn, mode='PDD')
intial_network = sim.run_sim()
wn_recovery.output_water_network(intial_network, wn, timestamp=23*3600, 
                                plotly_options={'title':'Initial Water Network',
                                 'figsize':[800, 450], 'node_cmap':'RdBu',
                             })

## Time t1: after hazard happens
### calculate water network damges

In [ ]:
client = InsecureIncoreClient("http://incore2-services:8888/", 'cwang138')
water_facility_shp = 'SEASIDE/skeletonized_WN/Facility_Skeletonized.shp'
water_pipeline_shp = 'SEASIDE/skeletonized_WN/Pipeline_Skeletonized.shp'
wn_dmg = WaterNetworkDamage(client, water_facility_shp, water_pipeline_shp)
wn_dmg.water_netowork_damage(
    water_facility_mapping_id='5b47c3b1337d4a387e85564a',
    water_pipeline_mapping_id='5ba55a2aec2309043530887c',
    hazard_id='5ba92505ec23090435209071',
    num_threads=0)

In [ ]:
pipe_PEDS = pd.read_csv('pipeline_dmg.csv', index_col=0)
pipe_PEDS.index = pipe_PEDS.index.map(str)

pump_PEDS = pd.read_csv('pump_dmg.csv', index_col=0)
pump_PEDS.index = pump_PEDS.index.map(str)

tank_PEDS = pd.read_csv('tank_dmg.csv', index_col=0)
tank_PEDS.index = tank_PEDS.index.map(str)

In [ ]:
pipe_PEDS.head()

In [ ]:
tank_PEDS.head()

In [ ]:
pump_PEDS.head()

### calculate the population dislocation

#### calculate building damage

In [ ]:
# seaside building inventory
bldg_dataset_id = ""
hazard_type = "earthquake"
hazard_id = "5ba92505ec23090435209071"
dmg_ratio_id = "5a284f2ec7d30d13bc08209a"
mapping_id = "5b47b350337d4a3629076f2c"

# set parameters and dataset
bldg_dmg = BuildingDamage(client)
bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
bldg_dmg.load_remote_input_dataset("dmg_ratios", dmg_ratio_id)

result_name = "seaside_bldg_dmg_result"
bldg_dmg.set_parameter("result_name", result_name)
bldg_dmg.set_parameter("mapping_id", mapping_id)
bldg_dmg.set_parameter("hazard_type", hazard_type)
bldg_dmg.set_parameter("hazard_id", hazard_id)
bldg_dmg.set_parameter("num_cpu", 1)

# Run Analysis
bldg_dmg.run_analysis()       

#### Start of the Stochastic Population Allocation

In [ ]:
output_file_path = ""

# Population Dislocation
podi = PopulationDislocation(client, output_file_path, False)
merged_block_inv = PopulationDislocationUtil.merge_damage_population_block(
    building_dmg_file='seaside_bldg_dmg_result.csv',
    population_allocation_file='pop_allocation_1111.csv',
    block_data_file='IN-CORE_01av3_SetupSeaside_FourInventories_2018-08-29_bgdata.csv')
merged_final_inv = podi.get_dislocation(seed_i, merged_block_inv)

# save to csv
merged_final_inv.to_csv(output_file_path + "final_inventory_" + str(seed_i) + ".csv", sep=",")

### calculate the change of demand

In [ ]:
waternode_population1 = \
WaterNetworkRecoveryUtil.calc_waternode_population('waternode_building_relations.csv', 
                                                   'final_inventory_1111.csv')
waternode_population1

In [ ]:
demand1 = WaterNetworkRecoveryUtil.calc_waternode_demand('waternode_building_relations.csv',waternode_population1)
demand1

In [ ]:
results1day, wn1day = wn_recovery.wn_impact1day(pipe_PEDS, pump_PEDS, tank_PEDS, demand=demand1, seed=2)

In [ ]:
for i in range(0, 24, 6):
    animate(i, results1day, wn1day)

## Time t2: Start Recvoery
### need to have seaside pipeline zoning file. Mock file right now

In [ ]:
WN_rec_atr, rec_params = \
wn_recovery.set_recovery_attributes(pipe_PEDS, 
                                    pipe_zone_file='SEASIDE/skeletonized_WN/original_PipeZones_seaside_skeleton.csv',
                                    work_hour_day=16, tzero=4, prod_param = (20, 4), 
                                    crew= [[6, 5, 7, 7, 1, 6],[4, 3, 4, 4, 1, 4],[4, 3, 4, 4, 1, 4]])
results3day, wn3day = wn_recovery.wn_recoveryNday(pipe_PEDS, pump_PEDS,tank_PEDS, WN_rec_atr,rec_params, n_days=3,
                                                  demand=demand1,save_model=True, seed=2)

In [ ]:
for i in range(17, 72, 24):
    animate(i, results3day, wn3day)

## Time t3: After Recovery, additional dislocation happens and water demand change

In [ ]:
waternode_population2 = WaterNetworkRecoveryUtil.additional_waternode_population_dislocation(timestep=7*3600, 
                                                                       wntr_sim_resultsNday=results3day,
                                                                       prev_waternode_population=waternode_population1)
waternode_population2

In [ ]:
demand2 = WaterNetworkRecoveryUtil.calc_waternode_demand('waternode_building_relations.csv', waternode_population2)
demand2

In [ ]:
results5day, wn5day = wn_recovery.wn_rec_intrp_Nday(n_days=5, intrp_day=3,demand=demand2)

In [ ]:
for i in range(17, 5*24, 24):
    animate(i, results5day, wn5day)

In [ ]:
results1day.node['pressure']